In [ ]:
import pandas as pd
import numpy as np
import os
import nsepy
import wget
from nsepy.derivatives import get_expiry_date
from nsepy import get_history
from datetime import datetime, timedelta
from urllib.request import Request, urlopen
import shutil 

df2014 = pd.read_csv("nifty2014.csv")
df2015 = pd.read_csv("nifty2015.csv")
df2016 = pd.read_csv("nifty2016.csv")
df2017 = pd.read_csv("nifty2017.csv")
df2018 = pd.read_csv("nifty2018.csv")
df2019 = pd.read_csv("nifty2019.csv")
df2020 = pd.read_csv("nifty2020.csv")
df2021 = pd.read_csv("nifty2021.csv")
dfin = df2014.append(df2015)
dfin = dfin.append(df2016)
dfin = dfin.append(df2017)
# dfin = df2016.append(df2017)
dfin = dfin.append(df2018)
dfin = dfin.append(df2019)
dfin = dfin.append(df2020)
dfin = dfin.append(df2021)
# dfin.drop(['Shares Traded','Close','Open','Turnover (Rs. Cr)'],inplace=True,axis=1)
# dfin['Close'] = (dfin['Low'] + dfin['High']) / 2
# dfin.drop(['Low','High'],inplace=True,axis=1)
# dfin['Gap'] = dfin['Open'] - dfin['Close'].shift(1)
# dfin['Gap'] = dfin['Gap'].fillna(0)
dfin.drop(['Shares Traded','Open','Low','High','Turnover (Rs. Cr)'],inplace=True,axis=1)
dfin.head()

In [ ]:
option_file = 'OptionPrice_dumpall.csv'

def load_option_prices():
    if os.path.exists(option_file):
        df = pd.read_csv(option_file)
        df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
        df['ExpiryDay'] = pd.to_datetime(df['ExpiryDay'], format='%Y-%m-%d')
    else:
        df = pd.DataFrame(columns=['Date', 'Name', 'Type', 'StrikePrice', 'ExpiryDay','Price'])
    return df  
options_df = load_option_prices()


def get_url(dt):
    base_url = "https://www1.nseindia.com/content/historical/DERIVATIVES/" #2020/FEB/fo06FEB2020bhav.csv.zip
    yr = dt.year
    day = dt.day
    mon = dt.strftime('%b').upper()
    filename = 'fo{2:02d}{1}{0}bhav.csv.zip'.format(yr,mon,day)
    return '{0}{1}/{2}/{3}'.format(base_url,yr,mon,filename), filename

def download_file(link, file_name, length=2048):
    ##downloding file from site
    cookie = "pointer=1; sym1=LT; ak_bmsc=2AB71B1FC0E935FDE369C91BFE1AF4481736535670770000A17ABC604B996D26~plPIUfgdZT+ZJm07sKl59OD9ixQnW5p3JmrGz6v2rFeuixdbGUpV32o1F5zDHVfXpMpgzlYbBJx7UZK7lcw+c8Um3D5LxZTa/vxqOnrAgAa5edWuS9k/nQC95LluH6n+bKoeHn97qWg8HwS65Wxp3BF1kfhwXoNW4VdwmeVDTt/quRBenX/TjJyuneXeQ5/XjiVRiDyG8+Rz39KJemmWH3auv1CaxBTrz0U9gyJ8Cow1ZL8E/NJtuv7Ry9JOzO8B4qnBoVnZ1iZpNJGASVuzl/jw==; AKA_A2=A; bm_sv=B599286690E667A7A7E93C8F87139702~SW01SHEikU6NqJY8hfA32ap6jbTJ7VGGCsVrrA+dyEORiqcOrCrYenWGWfE7VYepOZwN28tUW/gwnAYIK8c3HBWJPlC6w88AaklEUpf/zR02qHzXc0DfFTUeo6gWH3FOwG40M1l4Xksba+rZAQ7mzLutHTCyPu8QbzXA5CVQQzw=; bm_mi=BC64742DDFEDE631D8D91809CB778CEB~y7KMrh0/pgCdjI4fAb3o+o0rylyE+/XAo9OyQY+cd4uWxrhwJRs6BDyXaCkhQrlHAcyZfzFqmD1i0KycCwqAFJlJYRdNmvDkYSIANMYpRlkyVVNbQ2Wfd2YjQWRwQ3CUjqKcNDsMIblJoNo5Ikgv9Pjc2rL2zKdqG7n9qyyP8nTa5jFuX7yIhkphBOLO8ATlMPUKs4CFz+vy6/yn8EwgUCZpudtCPUNZNtLYcJ3fiKd1EYpjGprFEp/VHRsAc2X7GcraZNAf3iIPqaI2GsreZw==; NSE-TEST-1=1927290890.20480.0000";

    header = {
        'Accept-Encoding': 'gzip, deflate, sdch, br',
        'Accept-Language': 'fr-FR,fr;q=0.8,en-US;q=0.6,en;q=0.4',
        'Host': 'www1.nseindia.com',
        'Referer': 'https://www1.nseindia.com/',
        'User-Agent': 'Mozilla/5.0 (X11; Linux i686) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/53.0.2785.143 Chrome/53.0.2785.143 Safari/537.36',
        'X-Requested-With': 'XMLHttpRequest',
        "Cookie": cookie
    }
    
    try:
        req = Request(link, headers=header)
        with open(file_name, 'wb') as writer:
            request = urlopen(req, timeout=3)
            shutil.copyfileobj(request, writer, length)
#             print(f'{file} downloaded with success!')
    except Exception as e:
        print(f'{link} cannot be downloaded:', e)
        raise Exception

#         link = "https://www1.nseindia.com/content/historical/DERIVATIVES/2017/JAN/fo19MAR2014bhav.csv.zip"
# download_file(link,"fo19MAR2014bhav.csv.zip")

In [ ]:
# load_sample = "OPTIDX_NIFTY_PE_29-Nov-2017_TO_31-Dec-2017.csv"
# dfs = pd.read_csv(os.path.join("downloads",load_sample))
# dfs['Expiry'] = pd.to_datetime(dfs['Expiry'], format='%d-%b-%Y')
# dfs['Date'] = pd.to_datetime(dfs['Date'], format='%d-%b-%Y')
# dfs["Price"]=0
# dfs = dfs[['Date','Symbol',"Option Type","Strike Price","Expiry","Price","Close","Low","High","Open"]]
# dfs.rename(columns = {'Symbol':'Name', 'Option Type':'Type', 'Strike Price':'StrikePrice', 'Expiry':'ExpiryDay',
#                      "Close":"ClosePrice", "Low":"LowPrice", "High":"HighPrice","Open":"OpenPrice"}, inplace = True)

# dfs.head()
# options_df = options_df.append(dfs)
# options_df['ExpiryDay'] = pd.to_datetime(options_df['ExpiryDay'], format='%Y-%m-%d')
# options_df.to_csv(option_file, header=True, sep=',', index=False)

In [ ]:
options_df.tail()

In [ ]:
df = dfin.copy()
df['Date'] = df['Date'].apply(lambda x : datetime.strptime(x,"%d-%b-%Y"))
#df.head(20)

In [ ]:
df['Day'] = df['Date'].apply(lambda x: x.weekday())
df['Month'] = df['Date'].apply(lambda x: x.month)
df['MonthYear'] = df['Date'].apply(lambda x: str(x.year) + str(x.month))
df.head()

In [ ]:
def move_to_next_day(row):
    return_date = row['15day']
    if row['15day'].weekday() == 5:
        return_date = row['15day'] + timedelta(days=2)
    elif row['15day'].weekday() == 6:
        return_date = row['15day'] + timedelta(days=1)
    return return_date

In [ ]:
df_lastday = df.groupby(['MonthYear']).tail(1).copy().reset_index(drop=True)
window = 15
df_lastday['15day'] = df_lastday['Date'] + timedelta(days=window)
df_lastday['15day'] = df_lastday.apply(move_to_next_day,axis=1)
df_lastday['15dayWeekday'] = df_lastday.apply(lambda x : x['15day'].weekday(),axis=1)
df_lastday['row_num'] = df_lastday.index
df_lastday.head()

In [ ]:
df_15day = df[['Date','Close']].copy()
df_mid = pd.merge(df_lastday, df_15day, left_on=["15day"], right_on=["Date"], how="left")
df_mid.rename(columns = {'Date_x':'Date_Sell', 'Close_x':'Close_Sell', 'Date_y':'Date_Buy', 'Close_y':'Close_Buy'}, inplace = True)
df_mid.fillna(0, inplace=True)
df_err = df_mid[(df_mid['Close_Buy'] == 0)].copy()
# df_err["15daynew"] = df_err["15day"] + timedelta(days=1)
print(len(df_err))
df_mid.head()

In [ ]:
df_lastday.head()

In [ ]:
for index, row in df_err.iterrows():
#     print(row['row_num'], row['15day'])
    new_date = df[(df['Date'] >= row['15day'])].head(1)['Date']
#     print(row['row_num'], row['15day'], new_date)
    if list(new_date):
        df_lastday.at[row['row_num'],'15day'] = list(new_date)[0]
#     else:
#         df_lastday = df_lastday.drop(index=row['row_num'])
df_lastday.head()

In [ ]:
df_mid = pd.merge(df_lastday, df_15day, left_on=["15day"], right_on=["Date"], how="left")
df_mid.rename(columns = {'Date_x':'Date_Sell', 'Close_x':'Close_Sell', 'Date_y':'Date_Buy', 'Close_y':'Close_Buy'}, inplace = True)
df_mid.fillna(0, inplace=True)
df_err = df_mid[(df_mid['Close_Buy'] == 0)].copy()
if len(df_err) == 1:
    for index, row in df_err.iterrows():
        if len(df_mid) == row['row_num'] + 1:
            df_mid = df_mid.iloc[:-1 , :]
elif len(df_err) > 1:
    print(df_err.tail(10))
    print("Error")
else: print("No Errors")
# df_mid['Date_Buy'] = df_mid.apply(lambda x: x['Date_Buy'].date(), axis=1)
df_mid.tail()

In [ ]:
df_mid.drop(["Day","Month","15day","15dayWeekday","row_num"],axis=1,inplace=True)
df_mid['Year'] = df_mid['Date_Buy'].apply(lambda x: x.year)
df_mid['BeginYear'] = df_mid['Year'].diff(periods=-1)
df_mid.loc[df_mid.index[-1], 'BeginYear']= -1
df_mid.head()

In [ ]:
#df_lastday.tail(20)
# spread, profit_positive, profit_negative = 0, 3300, 6700
# OTM, spread, profit_positive, profit_negative= 0, 200, 3250, 10000
# OTM, spread, profit_positive, profit_negative = 200, 100, 1650, 5000 
# OTM, spread, profit_positive, profit_negative = 100, 200, 1350, 10000 
# OTM, spread, profit_positive, profit_negative = 200, 200, 2550, 10000 
# OTM, profit_positive, profit_negative = 100, 6250, -7500
OTM, profit_positive, profit_negative = 300, 2500, -5000
# OTM, profit_positive, profit_negative = 200, 5000, -5000

In [ ]:
dfsellput = df_mid.copy()
dfsellput['StrikePrice'] = dfsellput['Close_Sell'].apply(lambda x: round(x - OTM,-2))
# dfsellput['Movement'] = dfsellput['Close_Buy'] - dfsellput['StrikePrice']
# dfsellput['Profit'] = dfsellput.apply(lambda x: profit_positive if x['Movement'] > 0 else max(-profit_negative,x['Movement'] * 50),axis=1)
# dfsellput['Profit'] = dfsellput.apply(lambda x: x['Profit'] if x['Profit'] > 0 else x['Profit'] + profit_positive,axis=1)
# dfsellput['CumProfit'] = dfsellput['Profit'].cumsum()
# dfsellput['CumProfitPercentage'] = dfsellput['CumProfit'] / 115000dfsellput.head()

In [ ]:
def options_get_history(row, options_df):
    index_option = "NIFTY"
    put_strike_price = int(row['StrikePrice'])
    putopenprice, puthighprice, putlowprice, putcloseprice = 0,0,0,0
    found_price_put = options_df.loc[(options_df['Date'] == row['Date'].strftime("%Y-%m-%d")) & (options_df['Name'] == index_option) 
                                     & (options_df['Type'] == 'PE') & (options_df['StrikePrice'] == put_strike_price) 
                                     & (options_df['ExpiryDay'] == row['ExpiryDay'].strftime("%Y-%m-%d"))]
    if len(found_price_put) == 0: 
        
        #http://www1.nseindia.com/content/historical/DERIVATIVES/2020/FEB/fo06FEB2020bhav.csv.zip
        url, filename = get_url(row['Date'])
        print('Fetching Price from NSE dumps {0}_{1}_PE on {2}. FileName: {3}'.format(row['ExpiryDay'], put_strike_price, row['Date'].strftime("%Y-%m-%d"),url))
        op_file = os.path.join('downloads',filename)
        if not os.path.exists(op_file):
#             wget.download(url, out='./downloads')
            download_file(url,os.path.join("./downloads",filename))
#             print()
        df = pd.read_csv(op_file)
        df['EXPIRY_DT'] = pd.to_datetime(df['EXPIRY_DT'], format='%d-%b-%Y')
        df = df[df['INSTRUMENT'] == 'OPTIDX']
        df = df[df['SYMBOL'] == index_option]
        df = df[df['STRIKE_PR'] == put_strike_price]
        df = df[df['EXPIRY_DT'] == row['ExpiryDay'].strftime("%Y-%m-%d")]
        putopenprice = df[df['OPTION_TYP'] == 'PE']['OPEN'].values[0]
        puthighprice = df[df['OPTION_TYP'] == 'PE']['HIGH'].values[0]
        putlowprice = df[df['OPTION_TYP'] == 'PE']['LOW'].values[0]
        putcloseprice = df[df['OPTION_TYP'] == 'PE']['CLOSE'].values[0]
        temp_dict = {}
        temp_dict[row['Date']] = {'Name'          : index_option, 
                                    'Type'        : 'PE', 
                                    'StrikePrice' : put_strike_price,
                                    'ExpiryDay'   : row['ExpiryDay'],
                                    'ClosePrice'  : putcloseprice,
                                    'LowPrice'    : putlowprice,
                                    'HighPrice'   : puthighprice,
                                    'OpenPrice'   : putopenprice,
                                    }
        temp_df = pd.DataFrame.from_dict(temp_dict, orient='index')
        temp_df['Date'] = temp_df.index
        temp_df.reset_index(inplace=True,drop=True)
        options_df = options_df.append(temp_df, ignore_index = True,sort=False)
        
        temp_df = None 
    else:
        putopenprice = found_price_put['OpenPrice'].values[0]
        puthighprice = found_price_put['HighPrice'].values[0]
        putlowprice = found_price_put['LowPrice'].values[0]
        putcloseprice = found_price_put['ClosePrice'].values[0]
    return putopenprice, puthighprice, putlowprice, putcloseprice, options_df

In [ ]:
transputsellprice, transputhigh, transputlow, transputbuyprice, transputhighdate, transputlowdate = [],[],[],[],[],[]
for i, r in dfsellput.iterrows():
    expiry = list(sorted(get_expiry_date(year=r['Year'], month=r['Date'].month, stock=True)))[-1]
    if expiry == datetime.strptime("2014-02-27", "%Y-%m-%d").date(): expiry = expiry - timedelta(days=1)
    dfw = df[(df["Date"] >= r['Date_Sell']) & (df["Date"] <= r['Date_Buy'])][['Date','Close']]
    row = {
           "Date": r['Date_Sell'], 
           "Name": "NIFTY",
           "StrikePrice": r['StrikePrice'],
           "ExpiryDay": expiry
          }
    putopen, puthigh, putlow, putclose, put_date = [], [], [], [], []
    for ii, ir in dfw.iterrows():
        row['Date'] = ir['Date']
        putopenprice, puthighprice, putlowprice, putcloseprice, options_df = options_get_history(row, options_df)
        putopen.append(putopenprice)
        puthigh.append(puthighprice)
        putlow.append(putlowprice)
        putclose.append(putcloseprice)
        put_date.append(row['Date'])
#         print(puthighprice, putlowprice, putcloseprice)
    transputsellprice.append(putclose[0])
    transputbuyprice.append(putclose[-1])
    
    maxi = [(i,j.strftime("%Y-%m-%d")) for i,j in zip(puthigh[1:],put_date[1:]) if i >= putclose[0] * 1.5]
    if maxi:
        max_put, max_date = maxi[0]
    else:
        max_put, max_date= max(puthigh[1:]),0
    transputhigh.append(max_put)
    transputhighdate.append(max_date)
    lowe = [(i,j.strftime("%Y-%m-%d")) for i,j in zip(putlow[1:],put_date[1:]) if i <= putclose[0] * 0.5]
    if lowe:
        low_put, low_date = lowe[-1]
    else:
        low_put, low_date= max(putlow[1:]), 0
    transputlow.append(low_put)
    transputlowdate.append(low_date)

#     print()
#     print(transputsellprice, transputhigh, transputlow, transputbuyprice)
#     break
dfsellput['transputsellprice'] = transputsellprice
dfsellput['transputhigh'] = transputhigh
dfsellput['transputhighdate'] = transputhighdate
dfsellput['transputlow'] = transputlow
dfsellput['transputlowdate'] = transputlowdate
dfsellput['transputbuyprice'] = transputbuyprice
dfsellput.head()

In [ ]:
dfsellput['RiskPrice'] = dfsellput['transputsellprice'] * 1.5
dfsellput['StopLossHit'] = dfsellput.apply(lambda x: "Yes" if x['transputhigh'] - x['RiskPrice'] > 0 else "No", axis=1)
dfsellput['StopLossHitinDays'] = dfsellput.apply(lambda x: (datetime.strptime(x["transputhighdate"],"%Y-%m-%d") - x['Date_Sell']).days if x['StopLossHit'] == "Yes" else "0", axis=1) 
dfsellput['Profit'] = dfsellput.apply(lambda x: (x['transputsellprice'] - x['transputbuyprice']) * 50 if x['StopLossHit'] == 'No' else (x['transputsellprice'] - x['RiskPrice']) * 50 , axis=1)
dfsellput['SellMargin'] = dfsellput['Close_Sell'] * 6
dfsellput['ProfitPercentage'] = dfsellput['Profit'] / dfsellput['SellMargin']
dfsellput['CumProfit'] = dfsellput['Profit'].cumsum()
dfsellput['CumSellMargin'] = dfsellput['SellMargin'].cumsum()
dfsellput['row_num'] = dfsellput.index + 1
dfsellput['CumSellMargin'] = dfsellput['CumSellMargin']  / dfsellput['row_num']
dfsellput['CumProfitPercentage'] = dfsellput['CumProfit'] / dfsellput['CumSellMargin']

dfsellput.head()



In [ ]:
# min_low = []
# min_date = []
# gap = []
# gap_open = []
# gap_date = []
# for i, r in  dfsellput.iterrows():
#     dfw = df[(df["Date"] > r['Date_Sell']) & (df["Date"] <= r['Date_Buy'])][['Date','Close','Low','Gap','Open']]
#     dfw1 = dfw[(dfw['Low'] <= r['StrikePrice'])]
#     if len(dfw1):
#         cl = dfw1['Low'].max()
#         min_low.append(cl)
#         dt = dfw1[dfw1['Low'] == cl]
#         min_date.append(list(dt['Date'])[0])
#     else:
#         min_date.append(r['Date_Buy'])
#         min_low.append(r['Close_Buy'])
    
#     gcl = dfw['Gap'].min()

#     gap.append(gcl)
#     gdt = dfw[dfw['Gap'] == gcl]
#     gap_date.append(list(gdt['Date'])[0])
#     gap_open.append(list(gdt['Open'])[0])
    
# dfsellput['Min_Low'] = min_low   
# dfsellput['Min_Date'] = min_date
# dfsellput['Gap'] = gap
# dfsellput['Gap_Open'] = gap_open
# dfsellput['Gap_Date'] = gap_date
# dfsellput['Min_Days'] = (dfsellput['Min_Date'] - dfsellput['Date_Sell']).dt.days
# dfsellput['Pull_Down_Strike'] = dfsellput.apply(lambda x: x['Min_Low'] - x['StrikePrice'],axis=1)
# dfsellput['Exit_Diff'] = dfsellput['Close_Buy'] - dfsellput['StrikePrice']
# dfsellput['Profit'] = dfsellput.apply(lambda x: profit_positive if x['Pull_Down_Strike'] > 0 else profit_negative ,axis=1)
# dfsellput['Profit'] = dfsellput.apply(lambda x: profit_positive // 2 if (x['Min_Days'] >= 8 and x['Profit'] < 0 and x['Pull_Down_Strike'] > -100) else x['Profit'], axis=1)
# dfsellput['CumProfit'] = dfsellput['Profit'].cumsum()
# dfsellput['CumProfitPercentage'] = dfsellput['CumProfit'] / 100000
# dfsellput.head(5)

In [ ]:
# dfsellput.drop(['Gap','Gap_Open','Gap_Date','Open','Low','MonthYear','Year'],inplace=True,axis=1)
dfsellput.to_csv("nifty_sellput_optiondata_OTM" + str(OTM) +".csv")

In [ ]:
# options_df['ExpiryDay'] = options_df['ExpiryDay'].strftime("%Y-%m-%d")
options_df.drop_duplicates()
options_df.to_csv(option_file, header=True, sep=',', index=False)